In [ ]:
#| default_exp js

# Formatting Components

To expedite fast development, FastHTML comes with several built-in Javascript and formatting components.

In [ ]:
#| exporti
import re
from fastcore.utils import *
from fasthtml.xtend import Script,jsd,Style,Link

In [ ]:
#| export
def light_media(
        css: str # CSS to be included in the light media query
    ):
    "Render light media for day mode views"
    return Style('@media (prefers-color-scheme: light) {%s}' %css)

In [ ]:
light_media('.body {color: green;}')

```html
<style>@media (prefers-color-scheme: light) {.body {color: green;}}</style>

```

In [ ]:
#| export
def dark_media(
        css: str # CSS to be included in the dark media query
    ):
    "Render dark media for nught mode views"
    return Style('@media (prefers-color-scheme:  dark) {%s}' %css)

In [ ]:
dark_media('.body {color: white;}')

```html
<style>@media (prefers-color-scheme:  dark) {.body {color: white;}}</style>

```

In [ ]:
#| export
marked_imp = """import { marked } from "https://cdn.jsdelivr.net/npm/marked/lib/marked.esm.js";
    import { proc_htmx } from "https://cdn.jsdelivr.net/gh/answerdotai/fasthtml-js/fasthtml.js";
"""
def MarkdownJS(
        sel='.marked' # CSS selector for markdown elements
    ):
    """Implements browser-based markdown rendering. Usage example [here](/tutorials/quickstart_for_web_devs.html#rendering-markdown).
    
    """
    src = "proc_htmx('%s', e => e.innerHTML = marked.parse(e.textContent));" % sel
    return Script(marked_imp+src, type='module')

In [ ]:
MarkdownJS()

```html
<script type="module">import { marked } from "https://cdn.jsdelivr.net/npm/marked/lib/marked.esm.js";
    import { proc_htmx } from "https://cdn.jsdelivr.net/gh/answerdotai/fasthtml-js/fasthtml.js";
proc_htmx('.marked', e => e.innerHTML = marked.parse(e.textContent));</script>

```

In [ ]:
#| export
def KatexMarkdownJS(
        sel='.marked',  # CSS selector for markdown elements
        inline_delim='$',  # Delimiter for inline math
        display_delim='$$',  # Delimiter for long math
        math_envs=None  # List of environments to render as display math
    ):
    math_envs = math_envs or ['equation', 'align', 'gather', 'multline']
    env_list = ','.join(f"'{env}'" for env in math_envs)

    src = r"""
    import katex from "https://cdn.jsdelivr.net/npm/katex/dist/katex.mjs";
    const renderMath = (tex, displayMode) => { return katex.renderToString(tex, {
            throwOnError: false,
            displayMode: displayMode,
            output: 'html',
            trust: true
        });
    };
    const processLatexEnvironments = (content) => {
        return content.replace(/\\begin{(\w+)}([\s\S]*?)\\end{\1}/g, (match, env, innerContent) => {
            if ([%(env_list)s].includes(env)) { return `%(display_delim)s${match}%(display_delim)s`; }
            return match;
        });
    };
    proc_htmx('%(sel)s', e => {
        let content = processLatexEnvironments(e.textContent);
        // Handle display math (including environments)
        content = content.replace(/%(display_delim)s([\s\S]+?)%(display_delim)s/gm, (_, tex) => renderMath(tex.trim(), true));
        // Handle inline math
        content = content.replace(/(?<!\w)%(inline_delim)s([^%(inline_delim)s\s](?:[^%(inline_delim)s]*[^%(inline_delim)s\s])?)%(inline_delim)s(?!\w)/g, (_, tex) => renderMath(tex.trim(), false));
        e.innerHTML = marked.parse(content);
    });
    """
    format_dict = {
        'env_list': env_list,
        'sel': sel,
        'display_delim': re.escape(display_delim),
        'inline_delim': re.escape(inline_delim)
    }
    formatted_src = src % format_dict
    return (Script(marked_imp + formatted_src, type='module'),
            Link(rel="stylesheet", href="https://cdn.jsdelivr.net/npm/katex@0.16.11/dist/katex.min.css"))

KatexMarkdown usage example:

```python
from fasthtml.common import *

longexample = r"""
Long example:

$$\begin{array}{c}

\nabla \times \vec{\mathbf{B}} -\, \frac1c\, \frac{\partial\vec{\mathbf{E}}}{\partial t} &
= \frac{4\pi}{c}\vec{\mathbf{j}}    \nabla \cdot \vec{\mathbf{E}} & = 4 \pi \rho \\

\nabla \times \vec{\mathbf{E}}\, +\, \frac1c\, \frac{\partial\vec{\mathbf{B}}}{\partial t} & = \vec{\mathbf{0}} \\

\nabla \cdot \vec{\mathbf{B}} & = 0

\end{array}$$
"""

app, rt = fast_app(hdrs=[KatexMarkdownJS()])

@rt('/')
def get():
    return Titled("Katex Examples", 
        # Assigning the 'marked' class to any component renders the 
        # string content inside as markdown
        P(cls='marked')("Inline example: $\sqrt{3x-1}+(1+x)^2$"),
        Div(cls='marked')(longexample)
    )

serve()
```

In [ ]:
#| export
def HighlightJS(
        sel='pre code', # CSS selector for code elements. Default is industry standard, be careful before adjusting it
        langs:str|list|tuple='python',  # Language(s) to highlight
        light='atom-one-light',  # Light theme
        dark='atom-one-dark'  # Dark theme
    ):
    "Implements browser-based syntax highlighting. Usage example [here](/tutorials/quickstart_for_web_devs.html#code-highlighting)."
    src = """
hljs.addPlugin(new CopyButtonPlugin());
hljs.configure({'cssSelector': '%s'});
htmx.onLoad(hljs.highlightAll);""" % sel
    hjs = 'highlightjs','cdn-release', 'build'
    hjc = 'arronhunt'  ,'highlightjs-copy', 'dist'
    if isinstance(langs, str): langs = [langs]
    langjs = [jsd(*hjs, f'languages/{lang}.min.js') for lang in langs]
    return [jsd(*hjs, f'styles/{dark}.css', typ='css', media="(prefers-color-scheme: dark)"),
            jsd(*hjs, f'styles/{light}.css', typ='css', media="(prefers-color-scheme: light)"),
            jsd(*hjs, f'highlight.min.js'),
            jsd(*hjc, 'highlightjs-copy.min.js'),
            jsd(*hjc, 'highlightjs-copy.min.css', typ='css'),
            light_media('.hljs-copy-button {background-color: #2d2b57;}'),
            *langjs, Script(src, type='module')]

In [ ]:
#| export
def SortableJS(
        sel='.sortable',  # CSS selector for sortable elements
        ghost_class='blue-background-class'  # When an element is being dragged, this is the class used to distingish it from the rest
    ):
    src = """
import {Sortable} from 'https://cdn.jsdelivr.net/npm/sortablejs/+esm';
import {proc_htmx} from "https://cdn.jsdelivr.net/gh/answerdotai/fasthtml-js/fasthtml.js";
proc_htmx('%s', Sortable.create);
""" % sel
    return Script(src, type='module')

In [ ]:
SortableJS()

```html
<script type="module">
import {Sortable} from 'https://cdn.jsdelivr.net/npm/sortablejs/+esm';
import {proc_htmx} from "https://cdn.jsdelivr.net/gh/answerdotai/fasthtml-js/fasthtml.js";
proc_htmx('.sortable', Sortable.create);
</script>

```